In [1]:
from xclib.data import data_utils
import numpy as np
import time
import pickle

In [2]:
#Change this later. Make the path as command line argument
X_train = data_utils.read_sparse_file('ass3_parta_data/train_x.txt', force_header=True)
train_class = np.genfromtxt('ass3_parta_data/train_y.txt').reshape(-1, 1)

X_valid = data_utils.read_sparse_file('ass3_parta_data/valid_x.txt', force_header=True)
valid_class = np.genfromtxt('ass3_parta_data/valid_y.txt').reshape(-1, 1)

X_test = data_utils.read_sparse_file('ass3_parta_data/test_x.txt', force_header=True)
test_class = np.genfromtxt('ass3_parta_data/test_y.txt').reshape(-1, 1)

/home/ayushi/.local/lib/python3.6/site-packages/xclib-0.96-py3.6-linux-x86_64.egg/xclib/data/data_utils.py:173: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [3]:
f = open('ass3_parta_data/train_x.txt') 
m,n = f.readlines()[0].rstrip("\n").split(" ")
m,n = int(m),int(n)
f.close()

In [4]:
classes = np.unique(train_class)
pos_class_ind = np.where(train_class == 1)
neg_class_ind = np.where(train_class == 0)

In [5]:
attributes = np.array(range(X_train.A.shape[1]))

### Entropy H(Y) is calculated assuming Y only has two classes - 0 and 1

###### So we use log_2 and use pos and neg_num

In [6]:
def calc_entropyY(train_data, train_class):
    pos_num = train_data[np.where(train_class == 1)[0]].shape[0]
    neg_num = train_data[np.where(train_class == 0)[0]].shape[0]
    
    if (pos_num == 0) or (neg_num == 0):
        entr = 0 #in the limit
    else:
        entr = (pos_num/(pos_num+neg_num))*np.log2((pos_num+neg_num)/pos_num) + (neg_num/(pos_num+neg_num))*np.log2((pos_num+neg_num)/neg_num)
        
    return entr

In [7]:
#Entropy of Raw data
start = time.time()
print(calc_entropyY(X_train.A, train_class))
print("time = ", time.time()-start)

0.997775157550985
time =  0.11198925971984863


#### Reduction in Error is given by H(Y|X) by using value to split at the node
H(Y|X) 

       = H(Y|X=x)P(X=x)  -- find the two splits 

       = H(Y=y|X=x)P(X=x) -- find the entropy of both the splits. 

       = calc_entropy(split1, train_class_split1).P(X=x)
    

In [8]:
def calc_red_error(train_data, split, split_class, train_class):
    #print("In reduction of Error Module")
    #print(train_data.shape[0])
    error = 0
    for i in range(len(split)):
    #    print("Split shape = ", split[i].shape)
        error += (split[i].shape[0]/train_data.shape[0])*calc_entropyY(split[i], split_class[i])
    #print("Reduction in error = ", error)
    return error


In [9]:
def find_best_attr(train_data, train_class):
    #max_mutual_info = 0;   This doesnt work for the cases when mutual info becomes negative. 
    max_mutual_info = float("-inf");
    median = (np.median(train_data, axis=0))        
    #print(median)
    for i in range(len(attributes)):
        split = []
        split_class = []
        split.append(train_data[np.where(train_data[:,i] <= median[i])])
        split.append(train_data[np.where(train_data[:,i] > median[i])])
        split_class.append(train_class[np.where(train_data[:,i] <= median[i])])
        split_class.append(train_class[np.where(train_data[:,i] > median[i])])
        mutual_info = calc_entropyY(train_data, train_class) - calc_red_error(train_data, split, split_class, train_class)
        #print(mutual_info)
        if max_mutual_info < mutual_info:
            max_mutual_info = mutual_info
            best_attr = attributes[i]
            thres = median[i]
    return max_mutual_info, best_attr, thres

In [10]:
start = time.time()
print(find_best_attr(X_train.A, train_class))
print(time.time()-start)

(0.0793751756882577, 217, 0.0)
42.818392276763916


### Every Node  in the tree will have the following contents:

1. Left Subtree
2. Right Subtree
3. Attribute Name on which this node will split
4. Threshold of the attribute for left and right subtree (Median) - int value
5. Entropy
6. Whether this node is leaf or not - True/False
7. A final value of Y=0 or Y=1 in case of leaf

In [11]:
class newNode(object):
    def __init__(self, attr, Th, value):
        self.left = None
        self.right = None
        self.attribute = attr
        self.threshold = Th
        self.value = value
        self.leaf = False
        self.final = None
        self.parent = None
    
    def find_root(self):
        #root = self
        #print("current node = ",self)
        while (self.parent is not None):
            #print(self.parent)
            self = self.parent
            self = self.find_root()
        root = self
        #print("Root found at = ", root)
        return root

    def calc_accuracy(self, tra_data, tra_classes, val_data, val_classes, test_data, test_classes):
        root = self.find_root()
        print("Root found at = ", root)
        #print(root)
#         print("Hello")
        t_acc = 0; v_acc = 0; tes_acc = 0
        for i in range(len(tra_data)):
            pred = predict(root, tra_data[i])
            if (pred == tra_classes[i]):
                t_acc+=1
        for i in range(len(val_data)):
            pred = predict(root, val_data[i])
            if (pred == val_classes[i]):
                v_acc += 1
        for i in range(len(test_data)):
            pred = predict(root, test_data[i])
            if(pred == test_classes[i]):
                tes_acc+=1
        
        return t_acc/len(tra_classes), v_acc/len(val_classes), tes_acc/len(test_classes);

In [12]:
class Tree(object):
    def __init__(self):
        self.tree = None
    def buildTree(self, data, classes, parent):
        self.tree = growTree(data, classes, parent, 0)

In [13]:
def predict(tree, data):
    if tree is None:
        return -1
    if tree.leaf:
        return tree.final
    if tree.left is None and tree.right is None:
        if tree.value[0] > tree.value[1]:
            return 0
        else:
            return 1
    if data[tree.attribute] <= tree.threshold:
        #print(tree.parent)
        return predict(tree.left, data)
    elif data[tree.attribute] > tree.threshold:
        return predict(tree.right, data)


In [20]:
def growTree(train_data, classes, parent, depth, depth_threshold=20):
    global num_nodes, nodes
    global train_acc, val_acc, test_acc
    pos_num = train_data[np.where(classes == 1)[0]].shape[0]
    neg_num = train_data[np.where(classes == 0)[0]].shape[0]
    print("The number of nodes present in the graph = ", num_nodes)
    #print(neg_num, pos_num)

    if (depth == depth_threshold):
        num_nodes+=1
        leaf = newNode(None, None, None)
        leaf.parent = parent
        leaf.leaf = True
        if (pos_num > neg_num):
            leaf.final = 1
        else:
            leaf.final = 0
        return leaf
            
    elif(pos_num == 0 or neg_num == 0):
        num_nodes+=1
        leaf = newNode(None, None, None)
        leaf.leaf = True
        leaf.parent = parent
        
        if(pos_num != 0):
            leaf.final = 1
        elif(neg_num != 0):
            leaf.final = 0
        return leaf
        
    else:
        mutual_info, best_attr, thresh = find_best_attr(train_data, classes)
        print("Best Attribute with Threshold = ", best_attr, thresh)
        num_nodes+=1
        left_split = train_data[np.where(train_data[:,best_attr] <= thresh)]
        right_split = train_data[np.where(train_data[:,best_attr] > thresh)]
        left_split_class = classes[np.where(train_data[:,best_attr] <= thresh)]
        right_split_class = classes[np.where(train_data[:,best_attr] > thresh)]
        
        if (left_split.shape[0] == 0):
            leaf = newNode(None, None, None)
            leaf.parent = parent
            
            leaf.leaf = True
            pos = left_split[np.where(left_split_class == 1)[0]].shape[0]
            neg = left_split[np.where(left_split_class == 0)[0]].shape[0]
            if (pos>neg):
                leaf.final = 1
            else:
                leaf.final = 0
            return leaf
        elif (right_split.shape[0] == 0):
            leaf = newNode(None, None, None)
            leaf.leaf = True
            leaf.parent = parent
            
            pos = right_split[np.where(right_split_class == 1)[0]].shape[0]
            neg = right_split[np.where(right_split_class == 0)[0]].shape[0]
            if (pos>neg):
                leaf.final = 1
            else:
                leaf.final = 0
            return leaf
        else:
            addNode = newNode(best_attr, thresh, [neg_num, pos_num])
            addNode.parent = parent
            #print(parent)
            print("The value of the class examples in this node = ", addNode.value)
            if (num_nodes%50 == 1):
                nodes.append(num_nodes)
                t, v, tes = addNode.calc_accuracy(X_train.A, train_class, X_valid.A, valid_class, X_test.A, test_class)
                train_acc.append(t)
                val_acc.append(v)
                test_acc.append(tes)
            print("Building left tree on =", left_split.shape[0])
            addNode.left = growTree(left_split, left_split_class, addNode, depth+1)
            print("Building right tree in = ", right_split.shape[0])
            addNode.right = growTree(right_split, right_split_class, addNode, depth+1)
            return addNode

In [21]:
start = time.time()
num_nodes = 0
nodes = []
train_acc=[]; val_acc= []; test_acc = []
root = Tree()
#root = growTree(X_train.A, train_class, 0, 8)
root.buildTree(X_train.A, train_class, None)
print("Time Taken to grow tree = ", time.time() - start)

The number of nodes present in the graph =  0
Best Attribute with Threshold =  217 0.0
The value of the class examples in this node =  [30560, 34153]
Root found at =  <__main__.newNode object at 0x7f7ec5177a58>
Building left tree on = 47691
The number of nodes present in the graph =  1
Best Attribute with Threshold =  252 0.0
The value of the class examples in this node =  [17868, 29823]
Building left tree on = 43010
The number of nodes present in the graph =  2
Best Attribute with Threshold =  57 0.0
The value of the class examples in this node =  [17724, 25286]
Building left tree on = 39800
The number of nodes present in the graph =  3
Best Attribute with Threshold =  401 0.0
The value of the class examples in this node =  [17359, 22441]
Building left tree on = 37755
The number of nodes present in the graph =  4
Best Attribute with Threshold =  61 1.0
The value of the class examples in this node =  [17202, 20553]
Building left tree on = 19479
The number of nodes present in the graph 

Best Attribute with Threshold =  66 0.0
The value of the class examples in this node =  [29, 5]
Building left tree on = 32
The number of nodes present in the graph =  60
Best Attribute with Threshold =  20 0.0
The value of the class examples in this node =  [29, 3]
Building left tree on = 28
The number of nodes present in the graph =  61
Building right tree in =  4
The number of nodes present in the graph =  62
Building right tree in =  2
The number of nodes present in the graph =  63
Building right tree in =  4
The number of nodes present in the graph =  64
Building right tree in =  75
The number of nodes present in the graph =  65
Best Attribute with Threshold =  104 0.0
The value of the class examples in this node =  [67, 8]
Building left tree on = 74
The number of nodes present in the graph =  66
Best Attribute with Threshold =  115 4.0
The value of the class examples in this node =  [67, 7]
Building left tree on = 51
The number of nodes present in the graph =  67
Best Attribute wi

Best Attribute with Threshold =  22 4.0
The value of the class examples in this node =  [12, 10]
Building left tree on = 12
The number of nodes present in the graph =  128
Best Attribute with Threshold =  22 2.0
The value of the class examples in this node =  [9, 3]
Building left tree on = 6
The number of nodes present in the graph =  129
Best Attribute with Threshold =  34 1.0
The value of the class examples in this node =  [3, 3]
Building left tree on = 4
The number of nodes present in the graph =  130
Building right tree in =  2
The number of nodes present in the graph =  131
Building right tree in =  6
The number of nodes present in the graph =  132
Building right tree in =  10
The number of nodes present in the graph =  133
Best Attribute with Threshold =  151 43.0
The value of the class examples in this node =  [3, 7]
Building left tree on = 8
The number of nodes present in the graph =  134
Best Attribute with Threshold =  22 5.0
The value of the class examples in this node =  [1

Best Attribute with Threshold =  376 0.0
The value of the class examples in this node =  [25, 4]
Building left tree on = 28
The number of nodes present in the graph =  204
Best Attribute with Threshold =  203 4.0
The value of the class examples in this node =  [25, 3]
Building left tree on = 18
The number of nodes present in the graph =  205
Building right tree in =  10
The number of nodes present in the graph =  206
Building right tree in =  1
The number of nodes present in the graph =  207
Building right tree in =  2
The number of nodes present in the graph =  208
Building right tree in =  2
The number of nodes present in the graph =  209
Building right tree in =  268
The number of nodes present in the graph =  210
Best Attribute with Threshold =  138 92.0
The value of the class examples in this node =  [74, 194]
Building left tree on = 136
The number of nodes present in the graph =  211
Best Attribute with Threshold =  114 0.0
The value of the class examples in this node =  [65, 71]

Best Attribute with Threshold =  186 0.0
The value of the class examples in this node =  [424, 1199]
Building left tree on = 1464
The number of nodes present in the graph =  292
Best Attribute with Threshold =  90 0.0
The value of the class examples in this node =  [329, 1135]
Building left tree on = 915
The number of nodes present in the graph =  293
Best Attribute with Threshold =  27 0.0
The value of the class examples in this node =  [275, 640]
Building left tree on = 744
The number of nodes present in the graph =  294
Best Attribute with Threshold =  245 5.0
The value of the class examples in this node =  [175, 569]
Building left tree on = 376
The number of nodes present in the graph =  295
Best Attribute with Threshold =  100 1.0
The value of the class examples in this node =  [127, 249]
Building left tree on = 205
The number of nodes present in the graph =  296
Best Attribute with Threshold =  80 0.0
The value of the class examples in this node =  [103, 102]
Building left tree o

...

Best Attribute with Threshold =  34 144.0
The value of the class examples in this node =  [5, 8]
Building left tree on = 11
The number of nodes present in the graph =  8233
Best Attribute with Threshold =  115 42.0
The value of the class examples in this node =  [3, 8]
Building left tree on = 6
The number of nodes present in the graph =  8234
Building right tree in =  5
The number of nodes present in the graph =  8235
Best Attribute with Threshold =  203 74.0
The value of the class examples in this node =  [3, 2]
Building left tree on = 3
The number of nodes present in the graph =  8236
Building right tree in =  2
The number of nodes present in the graph =  8237
Building right tree in =  2
The number of nodes present in the graph =  8238
Building right tree in =  3
The number of nodes present in the graph =  8239
Building right tree in =  6
The number of nodes present in the graph =  8240
Building right tree in =  17
The number of nodes present in the graph =  8241
Building right tree 

Building left tree on = 3
The number of nodes present in the graph =  8301
Building right tree in =  3
The number of nodes present in the graph =  8302
Building right tree in =  58
The number of nodes present in the graph =  8303
Best Attribute with Threshold =  20 0.0
The value of the class examples in this node =  [47, 11]
Building left tree on = 55
The number of nodes present in the graph =  8304
Best Attribute with Threshold =  64 1.0
The value of the class examples in this node =  [47, 8]
Building left tree on = 38
The number of nodes present in the graph =  8305
Best Attribute with Threshold =  117 0.0
The value of the class examples in this node =  [30, 8]
Building left tree on = 25
The number of nodes present in the graph =  8306
Best Attribute with Threshold =  357 14.0
The value of the class examples in this node =  [17, 8]
Building left tree on = 22
The number of nodes present in the graph =  8307
Best Attribute with Threshold =  15 12.0
The value of the class examples in th

Best Attribute with Threshold =  245 22.0
The value of the class examples in this node =  [10, 5]
Building left tree on = 9
The number of nodes present in the graph =  8377
Best Attribute with Threshold =  115 45.0
The value of the class examples in this node =  [4, 5]
Building left tree on = 6
The number of nodes present in the graph =  8378
Best Attribute with Threshold =  15 12.0
The value of the class examples in this node =  [4, 2]
Building left tree on = 3
The number of nodes present in the graph =  8379
Best Attribute with Threshold =  24 19.0
The value of the class examples in this node =  [1, 2]
Building left tree on = 2
The number of nodes present in the graph =  8380
Building right tree in =  1
The number of nodes present in the graph =  8381
Building right tree in =  3
The number of nodes present in the graph =  8382
Building right tree in =  3
The number of nodes present in the graph =  8383
Building right tree in =  6
The number of nodes present in the graph =  8384
Build

The number of nodes present in the graph =  8455
Best Attribute with Threshold =  209 38.0
The value of the class examples in this node =  [1, 37]
Building left tree on = 26
The number of nodes present in the graph =  8456
Building right tree in =  12
The number of nodes present in the graph =  8457
Best Attribute with Threshold =  187 1120.0
The value of the class examples in this node =  [1, 11]
Building left tree on = 9
The number of nodes present in the graph =  8458
Building right tree in =  3
The number of nodes present in the graph =  8459
Best Attribute with Threshold =  36 2850.0
The value of the class examples in this node =  [1, 2]
Building left tree on = 2
The number of nodes present in the graph =  8460
Building right tree in =  1
The number of nodes present in the graph =  8461
Building right tree in =  27
The number of nodes present in the graph =  8462
Building right tree in =  2
The number of nodes present in the graph =  8463
Building right tree in =  50
The number of

Best Attribute with Threshold =  154 1.0
The value of the class examples in this node =  [4, 11]
Building left tree on = 12
The number of nodes present in the graph =  8522
Best Attribute with Threshold =  19 10.0
The value of the class examples in this node =  [4, 8]
Building left tree on = 9
The number of nodes present in the graph =  8523
Best Attribute with Threshold =  203 358.0
The value of the class examples in this node =  [2, 7]
Building left tree on = 6
The number of nodes present in the graph =  8524
Best Attribute with Threshold =  209 3.0
The value of the class examples in this node =  [2, 4]
Building left tree on = 3
The number of nodes present in the graph =  8525
Building right tree in =  3
The number of nodes present in the graph =  8526
Best Attribute with Threshold =  27 47.0
The value of the class examples in this node =  [2, 1]
Building left tree on = 2
The number of nodes present in the graph =  8527
Best Attribute with Threshold =  27 46.5
The value of the class 

In [22]:
print("The total number of nodes in the built tree = {}".format(num_nodes))

The total number of nodes in the built tree = 8597


In [26]:
train_acc, val_acc, test_acc = root.tree.calc_accuracy(X_train.A, train_class, X_valid.A, valid_class, X_test.A, test_class)
#val_acc = root.tree.calc_accuracy(X_valid.A, valid_class)
#test_acc = root.tree.calc_accuracy(X_test.A, test_class)

Root found at =  <__main__.newNode object at 0x7f7ec5177a58>


Test Accuracy = 71.67%
Train Accuracy = 
Validation Accuracy = 

In [27]:
print("The Training Accuracy on Full-Grown Tree is = {:2.3f}%".format(train_acc*100))
print("The Vaidation Accuracy on Full-Grown Tree is = {:2.3f}%".format(val_acc*100))
print("The Test Accuracy on Full-Grown Tree is = {:2.3f}%".format(test_acc*100))

The Training Accuracy on Full-Grown Tree is = 81.423%
The Vaidation Accuracy on Full-Grown Tree is = 75.056%
The Test Accuracy on Full-Grown Tree is = 75.504%


##### Depth = 3
The Training Accuracy on Full-Grown Tree is = 66.541%

The Vaidation Accuracy on Full-Grown Tree is = 66.345%

The Test Accuracy on Full-Grown Tree is = 66.715%

##### Depth = 5
The Training Accuracy on Full-Grown Tree is = 68.46846846846847%

The Vaidation Accuracy on Full-Grown Tree is = 67.85184498423882%

The Test Accuracy on Full-Grown Tree is = 68.41592879328728%

##### Depth = 8
The Training Accuracy on Full-Grown Tree is = 74.004%

The Vaidation Accuracy on Full-Grown Tree is = 72.872%

The Test Accuracy on Full-Grown Tree is = 73.488%

##### Dept = 15
The Training Accuracy on Full-Grown Tree is = 79.90511952776103%

The Vaidation Accuracy on Full-Grown Tree is = 76.08937511589097%

The Test Accuracy on Full-Grown Tree is = 76.63529739001437%

##### Depth = 18
The Training Accuracy on Full-Grown Tree is = 81.199%

The Vaidation Accuracy on Full-Grown Tree is = 75.672%

The Test Accuracy on Full-Grown Tree is = 76.121%

##### Depth = 20
The Training Accuracy on Full-Grown Tree is = 81.423%

The Vaidation Accuracy on Full-Grown Tree is = 75.056%

The Test Accuracy on Full-Grown Tree is = 75.504%

##### Depth = 30
The Training Accuracy on Full-Grown Tree is = 83.243%

The Vaidation Accuracy on Full-Grown Tree is = 72.270%

The Test Accuracy on Full-Grown Tree is = 72.050%

In [5]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, train_class)

In [6]:
tree.plot_tree(clf) 

[Text(319.05051113061876, 366.52, 'X[217] <= 1.5\ngini = 0.498\nsamples = 64713\nvalue = [30560, 34153]'),
 Text(257.36407037011065, 360.35999999999996, 'X[363] <= 31.5\ngini = 0.468\nsamples = 47727\nvalue = [17872, 29855]'),
 Text(225.01246732417226, 354.2, 'X[57] <= 0.5\ngini = 0.489\nsamples = 41489\nvalue = [17697, 23792]'),
 Text(174.68336964363738, 348.03999999999996, 'X[401] <= 0.5\ngini = 0.496\nsamples = 38291\nvalue = [17335, 20956]'),
 Text(88.80787074189956, 341.88, 'X[151] <= 8.5\ngini = 0.499\nsamples = 36235\nvalue = [17178, 19057]'),
 Text(34.19124282323241, 335.71999999999997, 'X[107] <= 13.5\ngini = 0.474\nsamples = 14650\nvalue = [5639, 9011]'),
 Text(21.498475631824196, 329.55999999999995, 'X[106] <= 0.5\ngini = 0.493\nsamples = 10045\nvalue = [4437, 5608]'),
 Text(19.75143691552121, 323.4, 'X[21] <= 0.5\ngini = 0.491\nsamples = 9804\nvalue = [4240, 5564]'),
 Text(17.7520326703668, 317.23999999999995, 'X[353] <= 0.5\ngini = 0.489\nsamples = 9692\nvalue = [4130, 556